# Ex. 3.2 HBASE
We use the *HappyBase* lib: https://happybase.readthedocs.io/en/latest/installation.html

In [ ]:
! conda install -c conda-forge -y happybase
#! pip install -c conda-forge  happybase

/bin/bash: conda: command not found


In [17]:
!pip install happybase

In [18]:
# imports
import happybase

In [19]:
# try to connect
server="ec2-34-254-248-152.eu-west-1.compute.amazonaws.com" #need to change this for every lab-session
connection = happybase.Connection(server)

In [20]:
families = {
    'cf1': dict(max_versions=10),
    'cf2': dict(max_versions=1, block_cache_enabled=False),
    'cf3': dict(),  # use defaults
}

In [ ]:
#connection.create_table('lab3', families)  # table allredy exists

In [21]:
print(connection.tables())

[b'49a5b1a_employees', b'dh_user_id_employees', b'employee_Group2', b'employee_Group_2', b'employee_group_five', b'employees_smunaret', b'group1_employee', b'group3_user_id_employees', b'lab3', b'mspitzna_employees', b'tm_user_id_employees', b'user_id_employees']


In [22]:
#connect to a table
table = connection.table('lab3')

In [23]:
#see column families
table.families()

{b'cf1': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 10,
  'name': b'cf1:',
  'time_to_live': 2147483647},
 b'cf2': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': b'cf2:',
  'time_to_live': 2147483647},
 b'cf3': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 3,
  'name': b'cf3:',
  'time_to_live': 2147483647}}

In [24]:
#insert something
table.put('row 6', {'cf1:col7': '17.33', 'cf1:col8': '344'})

In [25]:
table.families()

{b'cf1': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 10,
  'name': b'cf1:',
  'time_to_live': 2147483647},
 b'cf2': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 1,
  'name': b'cf2:',
  'time_to_live': 2147483647},
 b'cf3': {'block_cache_enabled': False,
  'bloom_filter_nb_hashes': 0,
  'bloom_filter_type': b'NONE',
  'bloom_filter_vector_size': 0,
  'compression': b'NONE',
  'in_memory': False,
  'max_versions': 3,
  'name': b'cf3:',
  'time_to_live': 2147483647}}

In [26]:
#read table
res=table.scan()

In [27]:
for key,row in res:
    print(key,row)

b'row 6' {b'cf1:col7': b'17.33', b'cf1:col8': b'344'}
b'row 7' {b'cf1:col7': b'17.33', b'cf1:col8': b'612'}


# Ex 1
Employee data in HBASE.
* create a new table to hold the employee data -> use a distinct name, like ``user_id_employees``
* How would you create the column families?
* read data from ``employees.csv`` and put it into the table ->  use  ``import csv`` to read the data -> [API docs](https://docs.python.org/3/library/csv.html)
* use batch-processing to enter more than one entry per transaction
* alter some entries of the table with ``timestamps``
* perform some sample scans for the latest entries

In [28]:
connection.create_table('dh_user_id_employees', families)
print(connection.tables())

AlreadyExists: ignored

In [29]:
families = {
    'cf1': dict(max_versions=10),
    'cf2': dict(max_versions=1, block_cache_enabled=False),
    'cf3': dict(),  # use defaults
}

In [32]:
import csv
with open('employees.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    i = 0;
    batch = table.batch(timestamp=1623419183)
    for row in reader:
      batch.put(row[0], {'cf1:col1': row[1], 'cf1:col2': row[2], 'cf1:col3': row[3], 'cf1:col4': row[4], 'cf2:col5': row[5]})
      i=i+1
      if i>100:
        break;
    batch.send()

In [34]:
#read table
res=table.scan(include_timestamp=True, timestamp=1623419183)
for key,row in res:
    print(key,row)